# 데이터 연결
~~~
recipes(output_recipes.csv) -> 레시피 데이터 원본
new(preferenec.csv) -> 재료 유무, vege/vegan, food type, healthiness
user_db(user_db1.csv) -> 사용자 데이터베이스, 재료 선호도

recipes(name, id, 식재료들) = new(name, id, 식재료들)

In [26]:
import pandas as pd

In [27]:
recipes = pd.read_csv('./output_recipes.csv')

new = pd.read_csv('./existence.csv')

user_db = pd.read_csv('./user_db1.csv')

## 이미지 분석으로 나온 데이터를 통한 필터링

In [28]:
def find_matching_recipes(ingredients, vegan_vege=False, dish_type=None, healthy=False):
    matching_rows = pd.DataFrame()

    for ingredient in ingredients:
        if ingredient in new.columns:
            matching_rows = new[new[ingredient] == 1]

    if vegan_vege:
        matching_rows = matching_rows[matching_rows['vegan/vege'].isin([True, False])]

    if dish_type:
        matching_rows = matching_rows[matching_rows['food type'].isin(dish_type)]

    if healthy:
        matching_rows = matching_rows[matching_rows['healthiness'].isin([True, False])]

    matching_rows.reset_index(drop=True, inplace=True)

    return matching_rows


In [29]:
ingredients = ['tomato', 'apple']

In [30]:
matching_rows = find_matching_recipes(ingredients)

In [31]:
matching_rows

,Unnamed: 0,name,id,apple,apricot,asparagus,avocado,bacon,banana,bean,...,sugar,sweet potato,tofu,tomato,watermelon,yogurt,zucchini,vege/vegan,food type,healthiness
0,5,apple a day milk shake,5289,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,False,dish,False
1,63,more more apple pear jigglers,63593,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,False,dish,False
2,175,cream of pumpkin or squash soup vegan,164526,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,False,dish,False
3,202,greens to go smoothie,353044,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,False,dish,False
4,240,nat s king cole coleslaw,164634,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,False,dish,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,229201,zillionth chicken salad recipe,192480,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,False,dish,False
1569,229447,zucchini and apple bulgur,38267,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,False,dish,False
1570,229448,zucchini and apple saute,247115,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,False,dish,False
1571,229598,zucchini bread with cream cheese filling,380850,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,False,dish,True


## 로그인, 가중치 계산으로 통한 5가지 요리(id) 리스트

In [32]:
def login(user_id_input, password_input):
    user_data = user_db[user_db['id'] == user_id_input]
    if not user_data.empty and user_data['password'].values[0] == password_input:
        return user_data
    return None

In [33]:
def find_top_matching_rows(user_data, matching_rows):

    common_ingredients = [col for col in user_data.columns if col not in ['id', 'password']]
    

    matching_rows['total_score'] = matching_rows[common_ingredients].mul(user_data[common_ingredients].values[0]).sum(axis=1)
    
    top_5_rows = matching_rows.nlargest(5, 'total_score')
    
    return top_5_rows['id'].tolist()

In [34]:
user_id_input = input("Enter your ID: ")
password_input = input("Enter your password: ")

user_data = login(user_id_input, password_input)

if user_data is not None:
    user_data = user_data.reset_index(drop=True)
    
    top_5_matching_ids = find_top_matching_rows(user_data, matching_rows)
    
    if top_5_matching_ids:
        print("Top 5 matching dish IDs:")
        print(top_5_matching_ids)
    else:
        print("No matching dishes found.")
else:
    print("Invalid credentials. Please try again.")


Top 5 matching dish IDs:
[118885, 380850, 247115, 38267, 192480]


/var/folders/jr/r32rns2x2dd3llvsz29fhf9m0000gn/T/ipykernel_29376/3367239443.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_rows['total_score'] = matching_rows[common_ingredients].mul(user_data[common_ingredients].values[0]).sum(axis=1)


## 5가지의 id를 원본 레시피 데이터에 연결

In [35]:
def find_matching_recipes(dish_ids, recipes):
    return recipes[recipes['id'].isin(dish_ids)][['name', 'minutes', 'ingredients', 'steps', 'description']]

In [36]:
if top_5_matching_ids:
    matching_recipes = find_matching_recipes(top_5_matching_ids, recipes)
    print("Matching Recipes:")
    print(matching_recipes)
else:
    print("No matching dishes found.")

Matching Recipes:
                                            name  minutes   
229201            zillionth chicken salad recipe       10  \
229447                 zucchini and apple bulgur       40   
229448                  zucchini and apple saute       35   
229598  zucchini bread with cream cheese filling      110   
229636                   zucchini carrot muffins       37   

                                              ingredients   
229201  ['cooked chicken breasts', 'onions', 'celery',...  \
229447  ['water', 'apple juice', 'bulgur', 'pine nuts'...   
229448  ['olive oil', 'red onion', 'apple', 'tomatoes'...   
229598  ['eggs', 'sugar', 'vanilla', 'oil', 'zucchini'...   
229636  ['carrots', 'zucchini', 'apple', 'flaked cocon...   

                                                    steps   
229201  ['mix it all together and refrigerate', 'best ...  \
229447  ['in a saucepan , bring water and apple juice ...   
229448  ['heat 2 tbs', 'oil in a frying pan , add the ...   
2295